In [ ]:
from bs4 import BeautifulSoup

# https://app.roll20.net/campaigns/chatarchive/8981579?p=1&onePage=true&hidewhispers=&hiderollresults=
# save page

# (pagenate https://app.roll20.net/campaigns/chatarchive/8981579?p=1 with cache?)
# annoying data seems to come from graphql responses
with open('data/Chat Log for Kingdom of Dust.html', mode='r', encoding="utf8") as fd:
    html_doc = fd.read()

soup = BeautifulSoup(html_doc, 'html.parser')

In [ ]:

messages = soup.find_all('div', 'message')

print(len(messages))



In [ ]:
from dateutil.parser import parse

current_time = None
timestamps = []
for message in messages:
    tstamp = message.find(class_='tstamp')
    if tstamp:
        current_time = parse(tstamp.string)
    timestamps.append(current_time)

print(timestamps[-20:])

In [ ]:
has_roll = []
for message in messages:
    has_roll.append(message.find(class_='inlinerollresult') is not None)

print(len([i for i in has_roll if i ]))

roll_timestamps = [timestamp for i, timestamp in zip(has_roll, timestamps) if i ]
roll_msgs = [message for i, message, in zip(has_roll, messages) if i ]



In [ ]:
import re
characters = []
roll_types = []
rolls = []
d20_rolls = []

roll_re = re.compile(r'">([0-9]+)</span>')

for message in roll_msgs:
    
    roll_group = message.find(class_='sheet-rolltemplate-rolls')

    character = roll_group.find(class_='sheet-charactername').string
    characters.append(character)
    roll_type = roll_group.find(class_='sheet-template-header').get_text().strip().replace('\n      ', ': ')
    roll_types.append(roll_type)

    this_rolls = []
    d20_roll = None

    for roll in roll_group.find_all('span', class_='inlinerollresult'):

        if 'original-title' in roll.attrs:
            roll_txt = roll['original-title']
        else:
            roll_txt = roll['title']
        this_rolls.append(roll_txt)
        if d20_roll is None and '1d20' in roll_txt:
            d20_roll = int(roll_re.search(roll_txt).group(1))

    rolls.append(this_rolls)
    d20_rolls.append(d20_roll)

print(characters[:10])
print(roll_types[:10])
print(d20_rolls[:10])


In [ ]:
from datetime import timedelta, datetime

MIN_ROLLS = 20
MAX_GAP = timedelta(hours=12)


start_time = roll_timestamps[0]
last_time = roll_timestamps[0]
count = 1

sessions = []

session_idx = []


for time in roll_timestamps:
    session_idx.append(len(sessions))
    if time - last_time > MAX_GAP:
        if count > MIN_ROLLS:
            sessions.append((start_time, last_time, count))
            print(last_time-start_time, count)
        count = 1
        start_time = time
    else:
        count += 1
    last_time = time
if count > MIN_ROLLS:
    sessions.append((start_time, last_time, count))
    print(last_time-start_time, count)

with open('data/sessions.txt', 'w') as fd:
    for session in sessions:
        fd.write(session[0].strftime('%Y-%m-%d\n'))


In [ ]:
roll_cats = [['skill'], ['strike'], ['saving'], ['initiative'], ['perception'], ['ability','Ability','abilities'], ['spell','cantrip']]


def map_func(roll_type):
    for vals in roll_cats:
        for val in vals:
            if val in roll_type:
                return vals[0]
    return 'other'

roll_types_simple = map(map_func, roll_types)





In [ ]:
import pandas as pd




roll_df = pd.DataFrame({
    'time': roll_timestamps,
    'character': characters,
    'session': session_idx,
    'type': roll_types_simple,
    'value': d20_rolls
})

roll_df = roll_df.dropna()

roll_df.to_csv('data/rolls.zip', index=False)

print(len(roll_df['character'].unique()))
print(len(roll_df['type'].unique()))
print(len(roll_df['session'].unique()))

print(roll_df['character'].unique())


In [ ]:
roll_df = pd.read_csv('data/rolls.zip', parse_dates=['time'])

print(roll_df.info())


In [ ]:
import plotly.express as px


last_session_df = roll_df[roll_df['time'] > datetime(2021,4,17)]

print(len(last_session_df))


arden_df =  last_session_df[last_session_df['character'] == 'Arden']


print(arden_df)

fig = px.histogram(last_session_df, x="value")
fig.show()


In [ ]:
counts = roll_df.groupby(['character']).count()['time'].sort_values()
pd.set_option("display.max_rows", None, "display.max_columns", None)
print(counts)



In [ ]:
counts = roll_df.groupby(['type']).count()['time'].sort_values()
print(counts)


In [ ]:
data = roll_df.groupby(['session','type']).count()['value'].reset_index(level=[0,1])
print(data)


In [ ]:


print(sum(roll_df['character'] == 'Raika '))

